In [64]:
import pandas as pd
import numpy as np

In [86]:
NUM_USERS = 100000
PROFESSOR_RATE = 0.7
STUDENT_RATE = 0.3

In [87]:
data = pd.read_csv('../scraping/results.csv')

In [88]:
data['student_count'] = data['student_count'] + 10
data['professor_count'] = data['professor_count'].clip(10)

In [89]:
data['student_rate'] = data['student_count'] / data['student_count'].sum()
data['professor_rate'] = data['professor_count'] / data['professor_count'].sum()

In [90]:
users = pd.DataFrame()

In [91]:
users['is_professor'] = np.random.choice([False, True], p = [1 - PROFESSOR_RATE, PROFESSOR_RATE], size = NUM_USERS)
users['is_student'] = np.random.choice([False, True], p = [1 - STUDENT_RATE, STUDENT_RATE], size = NUM_USERS)

In [92]:
num_professors = users['is_professor'].sum()
num_students = users['is_student'].sum()

In [93]:
users.loc[users['is_professor'] == True, 'teaching_subject'] = np.random.choice(data['subject_eng'].values, num_professors, p = data['professor_rate'])

users.loc[users['is_student'] == True, 'learning_subject'] = np.random.choice(data['subject_eng'].values, num_students, p = data['student_rate'])

In [94]:
users = users.dropna(subset = ['teaching_subject', 'learning_subject'], how = 'all')

In [95]:
users

,is_professor,is_student,teaching_subject,learning_subject
0,True,True,biology,religion
1,True,False,computer science,NaN
2,True,False,mathematics,NaN
3,True,False,German language,NaN
4,True,False,Romanian language,NaN
...,...,...,...,...
99995,True,False,chemistry,NaN
99996,True,True,biology,management
99997,True,False,English language,NaN
99998,True,False,English language,NaN
